In [ ]:
import math as math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.preprocessing import StandardScaler, Normalizer, KBinsDiscretizer, FunctionTransformer, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
data_cols = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]
target_col = "class"
col_names = ["id", *data_cols, target_col]

file_path = "../data/glass.data"
dataset = pd.read_csv(file_path, names=col_names)

# Exploration

In [ ]:
dataset.shape

In [ ]:
dataset.head(10)

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset[target_col].value_counts()

In [ ]:
dataset[data_cols].plot(kind="box", subplots=True, layout=(3, 3), sharex=False, sharey=False)
plt.tight_layout()
plt.show()

In [ ]:
pd.plotting.scatter_matrix(dataset[data_cols])
plt.show()

In [ ]:
sns.heatmap(dataset[data_cols].corr(), annot=True, fmt=".2f")

In [ ]:
melted = dataset[(np.abs(stats.zscore(dataset)) < 3).all(axis=1)]
melted = pd.melt(melted, target_col, data_cols)
sns.FacetGrid(
    melted,
    col="variable", hue=target_col, col_wrap=3,
    sharex=False, sharey=False
).map(sns.kdeplot, "value", fill=True, warn_singular=False).add_legend()

# Preprocessing

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
X_full = dataset[data_cols]
Y = dataset[target_col]

In [ ]:
missing_ratio = 0.05
X = X_full.mask(rng.random(X_full.shape) < missing_ratio)
X.info()

In [ ]:
test_size = 0.3
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, stratify=Y, random_state=rng)

Y_test.value_counts()

In [ ]:
imputers = {
    "KNN-1": KNNImputer(n_neighbors=1),
    "KNN-3": KNNImputer(n_neighbors=3),
    "MEAN": SimpleImputer(strategy="mean"),
    "MEDIAN": SimpleImputer(strategy="median")
}

In [ ]:
transformers = {
    "PASSTHROUGH": "passthrough",
    "SCALE": StandardScaler(),
    "NORMALIZE": Normalizer(),
    "DISCRETIZE-3": KBinsDiscretizer(n_bins=3, encode="onehot-dense"),
    "VAR-THRESHOLD-0.5": VarianceThreshold(threshold=0.5),
    "SELECT-3-BEST": SelectKBest(k=3),
    "PCA": PCA(random_state=rng),
    "ROBUST": RobustScaler(),
    "POWER": PowerTransformer(),
    "QUANTILE-UNIFORM": QuantileTransformer(n_quantiles=50, output_distribution="uniform", random_state=rng),
    "QUANTILE-NORMAL": QuantileTransformer(n_quantiles=50, output_distribution="normal", random_state=rng)
}

In [ ]:
sns.FacetGrid(
    pd.concat(
        pd.DataFrame(
            make_pipeline(transformers[n])
                .fit_transform(dataset[["Al", "Ca"]].to_numpy(), dataset[target_col]),
            columns=["Al", "Ca"]
        ).assign(**{"transformer": n, "class": dataset[target_col]})
        for n in ["PASSTHROUGH", "SCALE", "NORMALIZE", "ROBUST", "QUANTILE-UNIFORM", "QUANTILE-NORMAL"]
    ), col="transformer", col_wrap=3, sharex=False, sharey=False
).map_dataframe(sns.scatterplot, x="Al", y="Ca", hue="class", palette="bright").add_legend()

# Classification

In [ ]:
classifiers = {
    "NAIVE-BAYES-1e-9": GaussianNB(var_smoothing=1e-9),
    "NAIVE-BAYES-1e-6": GaussianNB(var_smoothing=1e-6),
    "NAIVE-BAYES-1e-3": GaussianNB(var_smoothing=1e-3),
    "DECISION-TREE-2-GINI": DecisionTreeClassifier(max_depth=2, criterion="gini", random_state=rng),
    "DECISION-TREE-10-GINI": DecisionTreeClassifier(max_depth=10, criterion="gini", random_state=rng),
    "DECISION-TREE-2-ENTROPY": DecisionTreeClassifier(max_depth=2, criterion="entropy", random_state=rng),
    "DECISION-TREE-10-ENTROPY": DecisionTreeClassifier(max_depth=10, criterion="entropy", random_state=rng),
    "KNN-3": KNeighborsClassifier(n_neighbors=3),
    "SVC-LINEAR": LinearSVC(random_state=rng),
    "SVC-RBF": SVC(gamma="auto", random_state=rng),
    "RANDOM-FOREST": RandomForestClassifier(random_state=rng)
}

In [ ]:
search_pipeline = Pipeline([
    ("imputer", "passthrough"),
    ("transformer", "passthrough"),
    ("classifier", next(iter(classifiers.values())))
])

In [ ]:
tree_classifier = make_pipeline(imputers["MEAN"], classifiers["DECISION-TREE-10-ENTROPY"])
tree_classifier.fit(X_train, Y_train)
plot_tree(tree_classifier.named_steps["decisiontreeclassifier"])
plt.show()

# Metrics

In [ ]:
grid_search = GridSearchCV(search_pipeline, {
    "imputer": list(imputers.values()),
    "transformer": list(transformers.values()),
    "classifier": list(classifiers.values())
}, n_jobs=-1)

grid_search.fit(X_train, Y_train)

print(grid_search.best_score_)
best_pipeline = grid_search.best_estimator_
best_pipeline

In [ ]:
def lookup(what, where):
    match where:
        case "imputer":
            d = imputers
        case "transformer":
            d = transformers
        case "classifier":
            d = classifiers
    return next(k for k in d if d[k] == what)

res = grid_search.cv_results_
for i, p in enumerate(grid_search.best_params_.keys()):
    x, y, std = zip(*sorted([
        max([
            (lookup(param, p), entry[1], entry[2])
            for entry in zip(res["params"], res["mean_test_score"], res["std_test_score"])
            if entry[0][p] == param
        ], key=lambda x: x[1])
        for param in grid_search.param_grid[p]
    ], key=lambda x: x[1]))

    ax = plt.axes()
    ax.barh(x, y, xerr=std)
    ax.set_xlabel(p)
    plt.show()

# Final test

In [ ]:
final_grid = GridSearchCV(
    make_pipeline(
        KNNImputer(n_neighbors=1),
        QuantileTransformer(random_state=rng),
        RandomForestClassifier(random_state=rng)
    ),
    {
        "quantiletransformer__n_quantiles": np.arange(1, 50, 5),
        "randomforestclassifier__n_estimators": np.arange(1, 150, 5)
    },
    n_jobs=-1
)

final_grid.fit(X_train, Y_train)
print(final_grid.best_score_)
final_estimator = final_grid.best_estimator_
final_estimator

In [ ]:
sns.lineplot(data=pd.DataFrame({
    "n_estimators": final_grid.cv_results_["param_randomforestclassifier__n_estimators"],
    "score": final_grid.cv_results_["mean_test_score"]
}), x="n_estimators", y="score", errorbar="se")

In [ ]:
sns.lineplot(data=pd.DataFrame({
    "n_quantiles": final_grid.cv_results_["param_quantiletransformer__n_quantiles"],
    "score": final_grid.cv_results_["mean_test_score"]
}), x="n_quantiles", y="score", errorbar="se")

In [ ]:
final_estimator.fit(X_train, Y_train)
predictions = final_estimator.predict(X_test)
ConfusionMatrixDisplay.from_predictions(Y_test, predictions)
plt.show()
print(accuracy_score(Y_test, predictions))
print(classification_report(Y_test, predictions))

### Data without missing values

In [ ]:
Xf_train, Xf_test, Yf_train, Yf_test = train_test_split(X_full, Y, test_size=test_size, stratify=Y, random_state=rng)

final_estimator.fit(Xf_train, Yf_train)
predictions = final_estimator.predict(Xf_test)
ConfusionMatrixDisplay.from_predictions(Yf_test, predictions)
plt.show()
print(accuracy_score(Yf_test, predictions))
print(classification_report(Yf_test, predictions))